# Analyzing IMDB Data in Keras

In [19]:
# Imports
import numpy as np
import keras
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras import regularizers
from keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping

%matplotlib inline

np.random.seed(42)

## 1. Loading the data
This dataset comes preloaded with Keras, so one simple command will get us training and testing data. There is a parameter for how many words we want to look at. We've set it at 1000, but feel free to experiment.

In [21]:
# Loading the data (it's preloaded in Keras)
(x_train, y_train), (x_test, y_test) = imdb.load_data(path="imdb.npz",
                                                      num_words=10000,
                                                      skip_top=30,
                                                      maxlen=None,
                                                      seed=113,
                                                      start_char=1,
                                                      oov_char=2,
                                                      index_from=3)

print(x_train.shape)
print(x_test.shape)

(25000,)
(25000,)


## 2. Examining the data
Notice that the data has been already pre-processed, where all the words have numbers, and the reviews come in as a vector with the words that the review contains. For example, if the word 'the' is the first one in our dictionary, and a review contains the word 'the', then there is a 1 in the corresponding vector.

The output comes as a vector of 1's and 0's, where 1 is a positive sentiment for the review, and 0 is negative.

In [22]:
print(y_train[0])

1


## 3. One-hot encoding the output
Here, we'll turn the input vectors into (0,1)-vectors. For example, if the pre-processed vector contains the number 14, then in the processed vector, the 14th entry will be 1.

In [23]:
# One-hot encoding the output into vector mode, each of length 1000
tokenizer = Tokenizer(num_words=10000)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
print(x_train[0])

[ 0.  0.  1. ...,  0.  0.  0.]


And we'll also one-hot encode the output.

In [24]:
x_train.shape

(25000, 10000)

In [25]:
# One-hot encoding the output
num_classes = 2
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_train.shape)
print(y_test.shape)

(25000, 2)
(25000, 2)


## 4. Building the  model architecture
Build a model here using sequential. Feel free to experiment with different layers and sizes! Also, experiment adding dropout to reduce overfitting.

In [27]:
# TODO: Build the model architecture
model = Sequential()
model.add(Dense(128, input_dim=x_train.shape[1], kernel_regularizer=regularizers.l2(0.02), activation='relu', name='first'))
model.add(Dropout(0.5))
model.add(Dense(64, input_dim=x_train.shape[1], kernel_regularizer=regularizers.l2(0.01), activation='relu', name='second'))
model.add(Dropout(0.3))
model.add(Dense(2, name='output'))
model.add(Activation('softmax'))

model.summary()

early_stopping = EarlyStopping(monitor='val_loss', patience=2)

# TODO: Compile the model using a loss function and an optimizer.
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
first (Dense)                (None, 128)               1280128   
_________________________________________________________________
dropout_11 (Dropout)         (None, 128)               0         
_________________________________________________________________
second (Dense)               (None, 64)                8256      
_________________________________________________________________
dropout_12 (Dropout)         (None, 64)                0         
_________________________________________________________________
output (Dense)               (None, 2)                 130       
_________________________________________________________________
activation_6 (Activation)    (None, 2)                 0         
Total params: 1,288,514
Trainable params: 1,288,514
Non-trainable params: 0
_________________________________________________________________


## 5. Training the model
Run the model here. Experiment with different batch_size, and number of epochs!

In [28]:
# TODO: Run the model. Feel free to experiment with different batch sizes and number of epochs.
model.fit(x_train, y_train, validation_split=0.15, epochs=10, batch_size=64,  callbacks=[early_stopping])


Train on 21250 samples, validate on 3750 samples
Epoch 1/10
21250/21250 [==============================] - 10s - loss: 1.0325 - acc: 0.8248 - val_loss: 0.6992 - val_acc: 0.8699

## 6. Evaluating the model
This will give you the accuracy of the model, as evaluated on the testing set. Can you get something over 85%?

In [29]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: ", score[1])

Accuracy:  0.8574
